# Exploring Wikipedia with Pandas

#### Import Libraries

In [ ]:
import pandas as pd
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

## Pageviews Data

 Create and explore a Dataframe.
 
 
 You may need to edit the path/file below.

In [ ]:
pageviews_df = pd.read_csv("pageviews-20160922-090000", delim_whitespace=True, header=None)

Let's look at the first 5 rows of the DF to make sure the dataframe looks correct.

In [ ]:
pageviews_df.head()

Let's add headers to our dataframe.

In [ ]:
pageviews_df.columns = ["project", "article", "requests", "bytes_served"]

In [ ]:
pageviews_df.tail()

In [ ]:
pageviews_df.sort_values('requests', ascending=False)[0:10]

This data is bad - bytes_served is 0 for each row. Let's go ahead and drop that column.

In [ ]:
pageviews_df = pageviews_df.drop('bytes_served', 1)

In [ ]:
pageviews_df = pageviews_df.drop('bytes_served', 1)

How many rows in the dataframe?

In [ ]:
len(pageviews_df.index)

In [ ]:
pageviews_df.count()

`.dtypes` displays the data types in each column

In [ ]:
pageviews_df.dtypes

Native Python Type | Pandas Type | Description
:---: | :---: | ---
string | object | most general dtype. assigned to columns with numbers and strings
int | int64 | numeric characters. 64 refers to the memory allocated to hold this character
float | float64 | numeric characters with decimals. columns with numbers and NaNs will default to this incase the missing values have decimals. 64 refers to the memory allocated to hold this character
N/A | datetime64, timedelta[ns] | values meant to hold time data

The `describe` method shows useful statistics about a dataframe. For this pageviews data, it's not very useful, but for more numeric data it gives a good snapshot.

In [ ]:
pageviews_df.describe()

#### Let's filter out only English Wikipedia projects

We need to create a new DF for only English projects

In [ ]:
ENpageviews_df = pageviews_df[pageviews_df['project'].str.contains("en")]

In [ ]:
ENpageviews_df.head()

In [ ]:
len(ENpageviews_df.index)

In [ ]:
ENpageviews_df['project'].unique()

In [ ]:
EN_clean_pageviews_df = ENpageviews_df[(ENpageviews_df['project'] != 'ten') & (ENpageviews_df['project'] != 'wg-en')]

In [ ]:
EN_clean_pageviews_df['project'].unique()

In [ ]:
EN_clean_pageviews_df.head(5)

* en -> English Wikipedia
* en.m -> English Wikipedia mobile
* en.d -> English dictionary
* en.n -> English news
* en.q -> English quotes

### Q) How much traffic did each English Wikipedia project receive in the past hour?

In [ ]:
EN_clean_pageviews_df.groupby('project').sum().sort_values('requests', ascending=False).head(10)

### Q) Which English articles receieved the most visits in the past hour?

In [ ]:
EN_clean_pageviews_df.sort_values('requests', ascending=False).head(20)

## Clickstream Data

 Create a new DF for clickstream data

In [ ]:
march_df = pd.read_csv("2016_03_clickstream.tsv", sep='\t')

In [ ]:
march_df.head()

In [ ]:
len(march_df.index)

In [ ]:
march_df.count()

Weird. Why do we get different answers using `len` and `count`?

In [ ]:
march_df[(march_df['curr'].isnull() == True)]

In [ ]:
len(march_df[(march_df['prev'].isnull() == True)])

So `len(pageviews_df.index)` and `pageviews_df.count()` are not interchangeable b/c `count` excludes NaNs ( it's slower)

Let's drop all the rows with `NaN` in any column. 

In [ ]:
march_df.dropna(inplace=True)

In [ ]:
march_df.count()

So, we've dropped 45 rows with missing data. 

However, there are other ways to deal with missing information. There's not a correct way to handle this... it all depends on the data and usecase. 

* drop rows with ALL NaNs
* fill NaNs with 0 (make a copy of the DF so you don't lose any data)
* foward fill values
* back fill values

In [ ]:
march_df.dtypes

### Q) How many articles recieved more than 1 million visits?

In [ ]:
march_df[march_df['n'] > 1000000].sort_values('n', ascending = False)

In [ ]:
len(march_df[march_df['n'] > 1000000])

### Q) Who were the top referers to Wikipedia?

In [ ]:
march_df.groupby('prev').sum().sort_values('n', ascending=False)[0:10]

* other-google -> google
* other-empty -> an empty referer (typically HTTPS traffic)
* other-bing -> bing
* other-wikipedia -> any Wikipedia page not in main English namespace
* other-other -> anything else
* other-yahoo -> yahoo
* other-twitter -> twitter
* Main_Page -> wikipedia home page

### Q) How many unique Wikipedia articles were visited?

In [ ]:
len(march_df['curr'].unique())

There are ~5 million English articles overall. This is telling us that ~4 million were visited AT LEAST once in March 2016. But this isn't entirely accurate...

### Q) How much traffic did Google send to the "Python (programming language)" article?

In [ ]:
march_df[(march_df['prev'] == "other-google") & (march_df['curr'] == 'Python_(programming_language)')]

### Q) What were the top 5 trending articles on Facebook?

In [ ]:
facebook_df = march_df[march_df['prev'] == 'other-facebook']
facebook_df.groupby('curr').sum().sort_values('n', ascending=False).head()

In [ ]:
facebook_df.groupby('curr').sum().sort_values('n', ascending=False).head().plot(kind='barh')
plt.show()

### Challenge: What were the top 5 trending articles on Twitter?

### Inflow vs. Outflow

Let's look at the ratio of inflow vs. outflow for the 5 most requested pages. 

First, let's find pageviews per article.

In [ ]:
inflow_df = march_df.groupby('curr').sum()
inflow_df.columns = ['in_count']

In [ ]:
inflow_df.head()

From inflow_df we can see that the "!!!" article received 7796 views in March 2016. !!! is actually a punk band out of California.

Now, let's find link clicks per article.

In [ ]:
outflow_df = march_df.groupby('prev').sum()
outflow_df.columns = ['out_count']

In [ ]:
outflow_df.head()

From outflow_df, we can know people clicked on links in the !!! article 2426 times to go onto another Wikipedia article.

Now, we'll join inflow_df and outflow_df and compare the ratio.

In [ ]:
in_out_df = inflow_df.join(outflow_df)
in_out_df['ratio'] = in_out_df['out_count'] / in_out_df['in_count']

In [ ]:
in_out_df.sort_values('in_count', ascending=False).head(10)

Only .002% of people who visited the Saint Patrick's Day page went on to another article. 

58% of people who went to Deaths in 2016 article continued to browse Wikpedia :( 

### Challenge:  find the pages with the highest ratio.

Wait... how can there be more links out of an article than into an article?

## Social Media + Politics

`pd.concat` is similar to SQL's union clause. It will vertically append objects to one another and combines columns with the same name.

In [ ]:
candidates = ['Donald_Trump', 'Hillary_Clinton']

In [ ]:
fb_df = pd.DataFrame()

for name in candidates:
    fb_df = pd.concat([fb_df, 
                      march_df[(march_df['curr'] == name) &
                               (march_df['prev'] == "other-facebook")]])

In [ ]:
fb_df.sort_values('n', ascending=False)

FYI- the cell below shows what would happen if we did not concat these DFs... not what we want.

In [ ]:
fb2_df = march_df[(march_df['curr'] == name) & (march_df['prev'] == "other-facebook")]
fb2_df

In [ ]:
twitter_df = pd.DataFrame()

for name in candidates:
    twitter_df = pd.concat([twitter_df, 
                      march_df[(march_df['curr'] == name) &
                               (march_df['prev'] == "other-twitter")]])

In [ ]:
twitter_df.sort_values('n', ascending=False)

In [ ]:
google_df = pd.DataFrame()

for name in candidates:
    google_df = pd.concat([google_df, 
                      march_df[(march_df['curr'] == name) &
                               (march_df['prev'] == "other-google")]])

In [ ]:
google_df.sort_values('n', ascending=False)

Skip down to the data viz below. 

### Challege: Find which articles the Donald Trump and Hillary Clinton articles sent the most traffic to. 

Hint #1: The answer is in the sankey diagram below, now you have to code it.

Hint #2: filter by prev equal to candidate and sort values

In [ ]:
%%html

<html>
  <head>
    <script type="text/javascript" src="https://www.gstatic.com/charts/loader.js"></script>
    <script type="text/javascript">
        google.charts.load('current', {'packages':['sankey']});
        google.charts.setOnLoadCallback(drawChart);

      function drawChart() {
        var data = new google.visualization.DataTable();
        data.addColumn('string', 'From');
        data.addColumn('string', 'To');
        data.addColumn('number', 'Weight');

        data.addRows([
          [ 'google', 'Trump', 3599213 ],
          [ 'google', 'Clinton', 781268 ],             
          [ 'facebook', 'Trump', 3024676 ],
          [ 'facebook', 'Clinton', 1638 ],
          [ 'twitter', 'Trump', 269423 ],
          [ 'twitter', "Clinton", 657 ],
          [ 'Trump', 'Melania Trump', 381387 ],
          [ 'Trump', 'Marla Maples', 250584 ], 
          [ 'Clinton', 'Bill Clinton', 49580 ],
          [ 'Clinton', 'Chelsea Clinton', 27585 ],                
        ]);

        // Sets chart options.
        var options = {
          width: 600,
        };

        // Instantiates and draws our chart, passing in some options.
        var chart = new google.visualization.Sankey(document.getElementById('sankey'));
        chart.draw(data, options);
      }
    </script>
  </head>
  <body>
    <div id="sankey" style="width: 900px; height: 300px;"></div>
  </body>
</html>